codecell 1: imports
codecell 2 : defining deterministic environment
codecell 3 : Running deterministic environment for 10 timesteps
codecell 4: defining stochastic environment
codecell 5 : running stochastic environment for 10 timestamps
codecell 6: defining Q learning 
codecell 7 : test_policy(ignore)
codecell 8:  Running Q Learnig for deterministic environment(it contains the plots replated to it)
codecell 9: defining run_greedy_policy
codecell 10: definining greedy policy and rendering 
code cell 11: runnin codecell 9 & 10 for deterministic environment
codecell 12 : defining gamma_tuning
codecell 13: running gamma tuning for deterministic env(contaains plots)

codecell 14 : defining alpha_tuning
codecell 15: running alpha tuning for deterministic env(contaains plots)
Codecell 16: defining SARSA
codecell 17: running sarsa for deterministic environment 
codecell 18: Running Q Learnig for stochastic  environment(it contains the plots related to it)
codecell 19: runnin codecell 9 & 10 for stochastic environment
codecell 20: running gamma tuning for Stochastic env(contaains plots)
codecell 21: running alpha  tuning for Stochastic env(contaains plots)
codecell 22: implementing SARSA for stochastic

In [ ]:
#CodeCell 1

import gymnasium as gym
import numpy as np
import random
from collections import defaultdict
import pickle
import matplotlib.pyplot as plt
import optuna

In [ ]:
#CodeCell 2
class DeterministicDroneDeliveryEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.grid_size = 6
        self.no_fly_zones = [(2,2), (2,3), (3,2), (3,3)]
        self.action_space = gym.spaces.Discrete(5)  # Up, Down, Left, Right, Pickup/Dropoff
        self.observation_space = gym.spaces.Tuple((
            gym.spaces.Box(0, 5, (2,), dtype=int),  # Drone position
            gym.spaces.Discrete(4),  # Carrying capacity (0-3 packages)
            gym.spaces.MultiBinary(3)  # Delivery status for each customer
        ))
        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.drone_pos = np.array([0, 0], dtype=int)
        self.carrying = 3
        self.num_customers = 3
        self.delivery_locations = [(1, 2), (4, 4), (5, 0)]  # Fixed delivery locations
        self.deliveries_completed = [False] * self.num_customers
        self.steps = 0
        return self._get_obs(), {}

    def step(self, action):
        self.steps += 1
        reward = -1  # Default step penalty
        terminated = False

        if action < 4:  # Movement
            new_pos = self.drone_pos.copy()
            if action == 0: new_pos[1] = min(5, new_pos[1]-1)  # Up
            elif action == 1: new_pos[1] = max(0, new_pos[1]+1)  # Down
            elif action == 2: new_pos[0] = max(0, new_pos[0]-1)  # Left
            elif action == 3: new_pos[0] = min(5, new_pos[0]+1)  # Right

            if self._is_valid(new_pos):
                self.drone_pos = new_pos
            else:
                reward = -100  # No-fly zone penalty

        elif action == 4:  # Pickup/Dropoff
            if np.array_equal(self.drone_pos, [0, 0]):  # At warehouse
                if self.carrying < 3:  # Pickup deliveries
                    pass
            else:
                for i, loc in enumerate(self.delivery_locations):
                    if np.array_equal(self.drone_pos, loc) and self.carrying > 0 and not self.deliveries_completed[i]:
                        self.deliveries_completed[i] = True
                        self.carrying -= 1
                        reward = 100
                        break

        # Check if all tasks are completed
        if all(self.deliveries_completed):
            terminated = True
            reward += 500  # Bonus for completing all tasks

        return self._get_obs(), reward, terminated, False, {}

    def _is_valid(self, pos):
        return (tuple(pos) not in self.no_fly_zones 
                and 0 <= pos[0] < self.grid_size 
                and 0 <= pos[1] < self.grid_size)

    def _get_obs(self):
        return (self.drone_pos.copy(), 
                self.carrying, 
                np.array(self.deliveries_completed))

    def render(self):
        grid = np.full((6,6), '⬜')
        for x,y in self.no_fly_zones:
            grid[y,x] = '🟥'
        
        for i, (x, y) in enumerate(self.delivery_locations):
            if not self.deliveries_completed[i]:
                grid[y,x] = f'📦{i+1}'
        
        grid[self.drone_pos[1], self.drone_pos[0]] = '🚁' #+ str(self.carrying)
        
        print('\n'.join([' '.join(row) for row in grid]))
        print(f"Carrying: {self.carrying} packages")
        print(f"Deliveries completed: {sum(self.deliveries_completed)}/{self.num_customers}")



In [ ]:
#CodeCell 3
# Example usage
env = DeterministicDroneDeliveryEnv()
obs, _ = env.reset()

print("Initial State:")
env.render()

for t in range(10):
    action = env.action_space.sample()
    next_obs, reward, terminated, truncated, _ = env.step(action)
    
    print(f"\nTimestep {t+1}")
    print(f"State: {obs}")
    print(f"Action: {['Up', 'Down', 'Left', 'Right', 'Pickup/Dropoff'][action]}")
    print(f"Reward: {reward}")
    env.render()
    
    obs = next_obs
    if terminated:
        print("Episode finished early.")
        break

print("Random agent simulation completed.")

In [ ]:
#CodeCell 4
class StochasticDroneDeliveryEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.grid_size = 6
        self.no_fly_zones = [(2,2), (2,3), (3,2), (3,3)]
        self.action_space = gym.spaces.Discrete(5)  # Up, Down, Left, Right, Pickup/Dropoff
        self.observation_space = gym.spaces.Tuple((
            gym.spaces.Box(0, 5, (2,), dtype=int),  # Drone position
            gym.spaces.Discrete(4),  # Carrying capacity (0-3 packages)
            gym.spaces.MultiBinary(3)  # Delivery status for each customer
        ))
        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.drone_pos = np.array([0, 0], dtype=int)
        self.carrying = 3
        self.num_customers = 3
        self.delivery_locations = [(1, 2), (4, 4), (5, 0)]  # Fixed delivery locations
        self.deliveries_completed = [False] * self.num_customers
        self.steps = 0
        return self._get_obs(), {}

    def step(self, action):
        self.steps += 1
        reward = -1  # Default step penalty
        terminated = False

        if action < 4:  # Movement
            new_pos = self.drone_pos.copy()
            if action == 0:  # Up
                if np.random.rand() < 0.9:
                    new_pos[1] = min(5, new_pos[1]-1)
                else:
                    new_pos = self._deviate(new_pos, action)
            elif action == 1:  # Down
                if np.random.rand() < 0.9:
                    new_pos[1] = max(0, new_pos[1]+1)
                else:
                    new_pos = self._deviate(new_pos, action)
            elif action == 2:  # Left
                if np.random.rand() < 0.9:
                    new_pos[0] = max(0, new_pos[0]-1)
                else:
                    new_pos = self._deviate(new_pos, action)
            elif action == 3:  # Right
                if np.random.rand() < 0.9:
                    new_pos[0] = min(5, new_pos[0]+1)
                else:
                    new_pos = self._deviate(new_pos, action)

            if self._is_valid(new_pos):
                self.drone_pos = new_pos
            else:
                reward = -100  # No-fly zone penalty

        elif action == 4:  # Pickup/Dropoff
            if np.array_equal(self.drone_pos, [0, 0]):  # At warehouse
                if self.carrying < 3:  # Pickup deliveries
                    pass
            else:
                for i, loc in enumerate(self.delivery_locations):
                    if np.array_equal(self.drone_pos, loc) and self.carrying > 0 and not self.deliveries_completed[i]:
                        self.deliveries_completed[i] = True
                        self.carrying -= 1
                        reward = 100
                        break

        # Check if all tasks are completed
        if all(self.deliveries_completed):
            terminated = True
            reward += 500  # Bonus for completing all tasks

        return self._get_obs(), reward, terminated, False, {}

    def _deviate(self, pos, action):
        if action == 0 or action == 1:  # Up or Down
            if np.random.rand() < 0.5:
                pos[0] = max(0, pos[0]-1)  # Deviate Left
            else:
                pos[0] = min(5, pos[0]+1)  # Deviate Right
        elif action == 2 or action == 3:  # Left or Right
            if np.random.rand() < 0.5:
                pos[1] = min(5, pos[1]+1)  # Deviate Down
            else:
                pos[1] = max(0, pos[1]-1)  # Deviate Up
        return pos

    def _is_valid(self, pos):
        return (tuple(pos) not in self.no_fly_zones 
                and 0 <= pos[0] < self.grid_size 
                and 0 <= pos[1] < self.grid_size)

    def _get_obs(self):
        return (self.drone_pos.copy(), 
                self.carrying, 
                np.array(self.deliveries_completed))

    def render(self):
        grid = np.full((6,6), '⬜')
        for x,y in self.no_fly_zones:
            grid[y,x] = '🟥'
        
        for i, (x, y) in enumerate(self.delivery_locations):
            if not self.deliveries_completed[i]:
                grid[y,x] = f'📦{i+1}'
        
        grid[self.drone_pos[1], self.drone_pos[0]] = '🚁' #+ str(self.carrying)
        
        print('\n'.join([' '.join(row) for row in grid]))
        print(f"Carrying: {self.carrying} packages")
        print(f"Deliveries completed: {sum(self.deliveries_completed)}/{self.num_customers}")

In [ ]:
#CodeCell 5
# Example usage
env = StochasticDroneDeliveryEnv()
obs, _ = env.reset()

print("Initial State:")
env.render()

for t in range(10):
    action = env.action_space.sample()
    next_obs, reward, terminated, truncated, _ = env.step(action)
    
    print(f"\nTimestep {t+1}")
    print(f"State: {obs}")
    print(f"Action: {['Up', 'Down', 'Left', 'Right', 'Pickup/Dropoff'][action]}")
    print(f"Reward: {reward}")
    env.render()
    
    obs = next_obs
    if terminated:
        print("Episode finished early.")
        break

print("Random agent simulation completed.")

⬜: Empty cell
🟥: No-fly zone
🚁: Drone (with carrying capacity)
📦: Package delivery location






In [ ]:
#CodeCell 6
#Q Learning implementation
# Initialize state space dimensions
np.set_printoptions(threshold=np.inf,  # Show all elements
                   linewidth=150,       # Wider display
                   precision=2,         # 2 decimal places
                   suppress=True)       # Don't use scientific notation
grid_size = 6
n_deliveries = 3
n_states = grid_size * grid_size * (2**n_deliveries)  # 288 states 
n_actions = 5

def state_to_index(state):
    
    pos, _, deliveries = state  # Ignore carrying value
    pos_idx = pos[0] * grid_size + pos[1]  # Convert 2D position to 1D (0-35)
    deliveries_idx = sum(int(d) << i for i, d in enumerate(deliveries))  # Binary encoding (0-7)
    return int(pos_idx + (grid_size * grid_size * deliveries_idx))

def q_learning(env, alpha=0.1, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, 
              min_epsilon=0.01, num_episodes=10, showplts=False, return_rewards=False):
    
    # Initialize Q-table with reduced state space
    q_table = np.zeros((n_states, n_actions), dtype=np.float32)
    print("Initial Q-table:")
    print(q_table)
    total_rewards = np.zeros(num_episodes, dtype=np.float32)
    epsilon_values = np.zeros(num_episodes, dtype=np.float32)
    
    try:
        for episode in range(num_episodes):
            state, _ = env.reset()
            state_idx = state_to_index(state)
            total_reward = 0
            done = False
            
            while not done:
                # Epsilon-greedy action selection
                if np.random.random() < epsilon:
                    action = env.action_space.sample()
                else:
                    action = int(np.argmax(q_table[state_idx]))
                
                # Take action
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                next_state_idx = state_to_index(next_state)
                
                # Q-learning update (vectorized)
                current_q = q_table[state_idx, action]
                next_max_q = np.max(q_table[next_state_idx])
                new_q = current_q + alpha * (reward + gamma * next_max_q - current_q)
                q_table[state_idx, action] = new_q
                #print("q_table updated")
                #print(reward)
                state_idx = next_state_idx
                total_reward += reward
            
            # Update tracking variables
            epsilon = max(min_epsilon, epsilon * epsilon_decay)
            total_rewards[episode] = total_reward
            epsilon_values[episode] = epsilon
            
            if (episode + 1) % 10 == 0:
                avg_reward = np.mean(total_rewards[max(0, episode-9):episode+1])
                print(f"Episode {episode+1}/{num_episodes}, Avg Reward: {avg_reward:.2f}, Epsilon: {epsilon:.3f}")
        
        if showplts:
            plot_training_results(total_rewards, epsilon_values, num_episodes)
        
        print("Full Q-table:")
        print(q_table)
        if return_rewards:
            return q_table, total_rewards
        return q_table
    
    except Exception as e:
        print(f"Training failed: {str(e)}")
        return None

def plot_training_results(total_rewards, epsilon_values, num_episodes):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot rewards
    ax1.plot(np.arange(num_episodes), total_rewards, 'b-')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Total Reward')
    ax1.set_title('Rewards per Episode')
    
    # Plot smoothed rewards
    window_size = min(10, num_episodes)
    smoothed_rewards = np.convolve(total_rewards, np.ones(window_size)/window_size, mode='valid')
    ax1.plot(np.arange(len(smoothed_rewards)), smoothed_rewards, 'r-', alpha=0.5, label='Smoothed')
    ax1.legend()
    
    # Plot epsilon decay
    ax2.plot(np.arange(num_episodes), epsilon_values, 'g-')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Epsilon Value')
    ax2.set_title('Epsilon Decay')
    
    plt.tight_layout()
    plt.show()

In [ ]:
#codeCell 7
def test_policy(env, q_table, num_episodes=100):
    total_rewards = 0

    for _ in range(num_episodes):
        state, _ = env.reset()
        terminated = False

        while not terminated:
            action = np.argmax(q_table[state])
            next_state, reward, terminated, truncated, _ = env.step(action)
            total_rewards += reward
            state = next_state

    avg_reward = total_rewards / num_episodes
    print(f"Average reward: {avg_reward}")
    return avg_reward

In [ ]:
#codeCell 8
env=DeterministicDroneDeliveryEnv()
q_table=q_learning(env,num_episodes=100,showplts=True)

# Save the Q-table/Policy table as a pickle file of the trained model
with open('deterministic_env_q_table.pkl', 'wb') as f:
    pickle.dump(q_table, f)


In [ ]:
#CodeCell 9

def run_greedy_policy(env, q_table, num_episodes=10):
   
    total_rewards = np.zeros(num_episodes, dtype=np.float32)

    for episode in range(num_episodes):
        state, _ = env.reset()
        state_idx = state_to_index(state)  # Convert state to index
        total_reward = 0
        terminated = False

        while not terminated:
            # Select action greedily from Q-table
            action = int(np.argmax(q_table[state_idx]))
            
            # Take action in environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            next_state_idx = state_to_index(next_state)
            
            total_reward += reward
            state_idx = next_state_idx

        total_rewards[episode] = total_reward

    # Visualization
    plt.figure(figsize=(8, 5))
    plt.plot(np.arange(num_episodes), total_rewards, 'bo-', label='Episode Reward')
    
    # Add moving average
    window_size = min(5, num_episodes)
    moving_avg = np.convolve(total_rewards, np.ones(window_size)/window_size, mode='valid')
    plt.plot(np.arange(len(moving_avg)), moving_avg, 'r-', label='Moving Average')
    
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Greedy Policy Evaluation Results')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    return total_rewards.tolist()  # Convert to list for compatibility

In [ ]:

# CodeCell 10
def run_greedy_policy_and_render(env, q_table, max_steps=100):
    
    state, _ = env.reset()
    state_idx = state_to_index(state)
    total_reward = 0
    step = 0
    terminated = False
    
    # Store trajectory for analysis
    trajectory = []
    action_names = ['Up', 'Down', 'Left', 'Right', 'Pickup/Dropoff']
    
    print("Initial State:")
    env.render()
    
    while not terminated and step < max_steps:
        # Get greedy action from Q-table
        action = int(np.argmax(q_table[state_idx]))
        trajectory.append((state, action))
        
        # Take action
        next_state, reward, terminated, truncated, _ = env.step(action)
        next_state_idx = state_to_index(next_state)
        
        # Update tracking variables
        total_reward += reward
        state = next_state
        state_idx = next_state_idx
        step += 1
        
        # Render environment state
        print(f"\nStep {step}")
        print(f"Action: {action_names[action]}")
        print(f"Reward: {reward}")
        print(f"Current Total Reward: {total_reward}")
        env.render()
    
    # Final statistics
    print("\nEpisode Summary:")
    print(f"Total Steps: {step}")
    print(f"Final Reward: {total_reward}")
    print(f"Terminated: {terminated}")
    
    if step >= max_steps:
        print("Warning: Episode reached maximum steps!")
    
    # Visualize trajectory
    '''
    if hasattr(env, 'grid_size'):
        plt.figure(figsize=(6, 6))
        plt.grid(True)
        plt.title('Agent Trajectory')
        
        # Plot trajectory
        traj_states = np.array([t[0][0] for t in trajectory])
        plt.plot(traj_states[:, 1], traj_states[:, 0], 'b-o', alpha=0.6)
        plt.plot(traj_states[0, 1], traj_states[0, 0], 'go', label='Start')
        plt.plot(traj_states[-1, 1], traj_states[-1, 0], 'ro', label='End')
        
        plt.xlim(-0.5, env.grid_size - 0.5)
        plt.ylim(-0.5, env.grid_size - 0.5)
        plt.legend()
        plt.show()
    '''
    
    return {
        'total_reward': total_reward,
        'steps': step,
        'trajectory': trajectory,
        'terminated': terminated
    }

In [ ]:
#codeCell 11
env=DeterministicDroneDeliveryEnv()
# Run the greedy policy and plot the results
run_greedy_policy(env, q_table, num_episodes=10)
# Run the greedy policy and render the results
run_greedy_policy_and_render(env, q_table)

In [ ]:
#codeCell 12
def gamma_tuning(env, num_episodes=100):
    
    # Select practical gamma values
    gamma_values = [0.9, 0.95, 0.99]  # Common values for discounted rewards
    results = {}
    
    for gamma in gamma_values:
        print(f"\nTraining with gamma = {gamma}")
        
        # Train Q-learning with current gamma
        q_table, training_rewards = q_learning(
            env=env,
            gamma=gamma,
            alpha=0.1,
            epsilon=1.0,
            epsilon_decay=0.995,
            min_epsilon=0.01,
            num_episodes=num_episodes,
            showplts=True,
            return_rewards=True
        )
        #print(f"Final Q-table : {q_table}")
        # Run greedy policy
        
        
        results[gamma] = {
            'q_table': q_table,
            'mean_reward': np.mean(training_rewards[-10:]),  # Last 10 episodes
            'std_reward': np.std(training_rewards[-10:])
        }
    run_greedy_policy(env, q_table, num_episodes=10)
    # Visualization
    plt.figure(figsize=(10, 6))
    
    # Plot mean rewards with error bars
    gammas = list(results.keys())
    means = [results[g]['mean_reward'] for g in gammas]
    stds = [results[g]['std_reward'] for g in gammas]
    
    plt.errorbar(gammas, means, yerr=stds, fmt='bo-', capsize=5, 
                label='Mean Reward ± Std')
    
    plt.xlabel('Gamma Value')
    plt.ylabel('Average Reward')
    plt.title('Q-Learning Performance vs Gamma')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    # Print results
    print("\nResults Summary:")
    for gamma in gamma_values:
        print(f"\nGamma = {gamma}")
        print(f"Mean Reward: {results[gamma]['mean_reward']:.2f}")
        print(f"Std Reward: {results[gamma]['std_reward']:.2f}")
    
    # Return best gamma
    best_gamma = max(results.keys(), key=lambda g: results[g]['mean_reward'])
    print(f"\nBest gamma value: {best_gamma}")
    
    return results

# Example usage:
# results = gamma_tuning(env)

In [ ]:
#codeCell 13
env= DeterministicDroneDeliveryEnv()
results = gamma_tuning(env)


In [ ]:
#codeCell 14
def epsilon_decay_tuning(env, num_episodes=100):
    """
    Evaluate Q-learning performance with different epsilon decay values
    
    Args:
        env: Gymnasium environment
        num_episodes: Number of episodes for each epsilon decay evaluation
    Returns:
        dict: Results for each epsilon decay value
    """
    # Select practical epsilon decay values
    epsilon_decay_values = [0.99, 0.995, 0.999]  # Common values for epsilon decay
    results = {}
    
    for epsilon_decay in epsilon_decay_values:
        print(f"\nTraining with epsilon decay = {epsilon_decay}")
        
        # Train Q-learning with current epsilon decay
        q_table, training_rewards = q_learning(
            env=env,
            gamma=0.99,
            alpha=0.1,
            epsilon=1.0,
            epsilon_decay=epsilon_decay,
            min_epsilon=0.01,
            num_episodes=num_episodes,
            showplts=True,
            return_rewards=True
        )
        
        # Run greedy policy
        run_greedy_policy(env, q_table, num_episodes=10)
        
        results[epsilon_decay] = {
            'q_table': q_table,
            'mean_reward': np.mean(training_rewards[-10:]),  # Last 10 episodes
            'std_reward': np.std(training_rewards[-10:])
        }
    
    # Visualization
    plt.figure(figsize=(10, 6))
    
    # Plot mean rewards with error bars
    decays = list(results.keys())
    means = [results[d]['mean_reward'] for d in decays]
    stds = [results[d]['std_reward'] for d in decays]
    
    plt.errorbar(decays, means, yerr=stds, fmt='bo-', capsize=5, 
                label='Mean Reward ± Std')
    
    plt.xlabel('Epsilon Decay Value')
    plt.ylabel('Average Reward')
    plt.title('Q-Learning Performance vs Epsilon Decay')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    # Print results
    print("\nResults Summary:")
    for epsilon_decay in epsilon_decay_values:
        print(f"\nEpsilon Decay = {epsilon_decay}")
        print(f"Mean Reward: {results[epsilon_decay]['mean_reward']:.2f}")
        print(f"Std Reward: {results[epsilon_decay]['std_reward']:.2f}")
    
    # Return best epsilon decay
    best_decay = max(results.keys(), key=lambda d: results[d]['mean_reward'])
    print(f"\nBest epsilon decay value: {best_decay}")
    
    return results


In [ ]:
#codeCell 15
env= DeterministicDroneDeliveryEnv()
results = epsilon_decay_tuning(env)

In [ ]:
#codeCell 16
def sarsa(env, alpha=0.1, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, 
          min_epsilon=0.01, num_episodes=10, showplts=False, return_rewards=False):
   
    # Initialize Q-table
    q_table = np.zeros((n_states, n_actions), dtype=np.float32)
    print("Initial Q-table:")
    print(q_table)
    total_rewards = np.zeros(num_episodes, dtype=np.float32)
    epsilon_values = np.zeros(num_episodes, dtype=np.float32)
    
    try:
        for episode in range(num_episodes):
            state, _ = env.reset()
            state_idx = state_to_index(state)
            total_reward = 0
            done = False
            
            # Select first action using epsilon-greedy
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = int(np.argmax(q_table[state_idx]))
            
            while not done:
                # Take action
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                next_state_idx = state_to_index(next_state)
                
                # Select next action using epsilon-greedy (SARSA is on-policy)
                if np.random.random() < epsilon:
                    next_action = env.action_space.sample()
                else:
                    next_action = int(np.argmax(q_table[next_state_idx]))
                
                # SARSA update
                current_q = q_table[state_idx, action]
                next_q = q_table[next_state_idx, next_action]
                q_table[state_idx, action] = current_q + alpha * (
                    reward + gamma * next_q - current_q
                )
                
                # Update state and action
                state_idx = next_state_idx
                action = next_action
                total_reward += reward
            
            # Update tracking variables
            epsilon = max(min_epsilon, epsilon * epsilon_decay)
            total_rewards[episode] = total_reward
            epsilon_values[episode] = epsilon
            
            if (episode + 1) % 10 == 0:
                avg_reward = np.mean(total_rewards[max(0, episode-9):episode+1])
                print(f"Episode {episode+1}/{num_episodes}, Avg Reward: {avg_reward:.2f}, Epsilon: {epsilon:.3f}")
        
        if showplts:
            plot_training_results(total_rewards, epsilon_values, num_episodes)
        print("Full Q-table:")
        print(q_table)
        if return_rewards:
            return q_table, total_rewards
        return q_table
    
    except Exception as e:
        print(f"Training failed: {str(e)}")
        return None

In [ ]:
# codeCell 17
env = DeterministicDroneDeliveryEnv()
q_table = sarsa(env, num_episodes=100, showplts=True)

In [ ]:
#codeCell 18
env=StochasticDroneDeliveryEnv()
q_table=q_learning(env,num_episodes=100,showplts=True)

# Save the Q-table/Policy table as a pickle file of the trained model
with open('stochastic_env_q_table.pkl', 'wb') as f:
    pickle.dump(q_table, f)

In [ ]:
#codeCell 19
env=StochasticDroneDeliveryEnv()
# Run the greedy policy and plot the results
run_greedy_policy(env, q_table, num_episodes=10)
# Run the greedy policy and render the results
run_greedy_policy_and_render(env, q_table)

In [ ]:
#codeCell 20
env= StochasticDroneDeliveryEnv()
results = gamma_tuning(env)

In [ ]:
#codeCell 21
env= StochasticDroneDeliveryEnv()
results = epsilon_decay_tuning(env)  

In [ ]:
#codeCell 22
env = StochasticDroneDeliveryEnv()
q_table = sarsa(env, num_episodes=100, showplts=True)
